In [2]:
import pandas as pan
import numpy as np

from statsmodels.stats.multitest import multipletests
from scipy.stats import ranksums


In [ ]:
### select strongest cis eqtl ### 
## run the findr 1_select_cis_eqtls.py## 

In [ ]:
#### prepare genotype data #############
# genotype prepared by writing the 2_prepare_genotype_data.py file#### 
### the output is genotypes_binary_strongest_eqtl.csv file ########

In [23]:
## reorder expression data ### 
## run the findr 3_reorder_expression_data.py ### 

# Reorder the gene expression data matrices:
#    Load data:
#    Reordering:


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


       numbers_of_genes:  [2836, 5720, 2884]
       genes with eQTLs:  2884
       all genes:  5720
    Check if all expected columns are present: True
    number of columns selected: 5720


In [2]:
#yeastract= pan.read_csv("./../data/input/yeastract/DNABinding_AND_Expr_TF_act_OR_inh_RegulationMatrix_Documented_2020511.csv.gz",header=0,index_col=0,
                           # delimiter=';')

In [6]:
## read yeastract file ######
yeastract= pan.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/data/input/yeastract/DNABinding_AND_Expr_TF_act_OR_inh_RegulationMatrix_Documented_2020511.csv.gz",header=0,index_col=0,
                            delimiter=';')

In [7]:
sum(yeastract.sum())

11486

In [ ]:
#### ensemble file downloaded and preprocessing done using the shell script file ##### 
## ensembl 109 used for yeast ## 

In [8]:
print('#  Selecting compatible subset of yeast regulatory network from yeastract with respect to elife paper.')
print('#  1. Load ensembl yeast gene library:')

#  Selecting compatible subset of yeast regulatory network from yeastract with respect to elife paper.
#  1. Load ensembl yeast gene library:


In [10]:
df_full = pan.read_csv( '/data/users/cs20s037/CITNonLinear/findr/findrfiles/data/cleaned_genes_ensembl109_yeast_step1.txt', skiprows=0,
                    names='chromosome position gene_str'.split(), 
                    usecols=[0,3,8],
                    comment='#',
                    sep='\t| ', engine='python')

df_full['id'] = df_full['gene_str'].str.split(':').str[1]
full_gene_list = df_full['id'].to_list()

In [11]:
#   The file used below only contains those entries with Name and ID.
df = pan.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/data/cleaned_genes_ensembl109_yeast_step2.txt", skiprows=0,
                    names='chromosome position gene_str name_str'.split(), 
                    usecols=[0,3,8,9],
                    comment='#',
                    sep='\t| ', engine='python')

In [12]:
print('#  data frame as loaded')
print(df)

#  data frame as loaded
     chromosome  position           gene_str   name_str
0             I      1807    ID=gene:YAL068C  Name=PAU8
1             I      7235    ID=gene:YAL067C  Name=SEO1
2             I     13363  ID=gene:YAL064C-A  Name=TDA8
3             I     24000    ID=gene:YAL063C  Name=FLO9
4             I     31567    ID=gene:YAL062W  Name=GDH3
...         ...       ...                ...        ...
5155        XVI    924304    ID=gene:YPR194C  Name=OPT2
5156        XVI    934034    ID=gene:YPR198W  Name=SGE1
5157        XVI    938148    ID=gene:YPR199C  Name=ARR1
5158        XVI    939279    ID=gene:YPR200C  Name=ARR2
5159        XVI    939922    ID=gene:YPR201W  Name=ARR3

[5160 rows x 4 columns]


In [13]:
#   Create new column name and id
df['name'] = df['name_str'].str.split('=').str[1]
df['id'] = df['gene_str'].str.split(':').str[1]
#   Sort it
dfs=df.sort_values(axis=0,by='chromosome position'.split() )
#   Make gene_name_dict
print('#  data frame after column operations and sorting')
print(dfs)
df['name'] = df['name'].str.upper()
df['id'] = df['id'].str.upper()
dfi = dfs.set_index('name')
gene_name_dict = dfi['id'].to_dict()
print('#  number of genes:', len(gene_name_dict))

#  data frame after column operations and sorting
     chromosome  position           gene_str   name_str  name         id
0             I      1807    ID=gene:YAL068C  Name=PAU8  PAU8    YAL068C
1             I      7235    ID=gene:YAL067C  Name=SEO1  SEO1    YAL067C
2             I     13363  ID=gene:YAL064C-A  Name=TDA8  TDA8  YAL064C-A
3             I     24000    ID=gene:YAL063C  Name=FLO9  FLO9    YAL063C
4             I     31567    ID=gene:YAL062W  Name=GDH3  GDH3    YAL062W
...         ...       ...                ...        ...   ...        ...
5155        XVI    924304    ID=gene:YPR194C  Name=OPT2  OPT2    YPR194C
5156        XVI    934034    ID=gene:YPR198W  Name=SGE1  SGE1    YPR198W
5157        XVI    938148    ID=gene:YPR199C  Name=ARR1  ARR1    YPR199C
5158        XVI    939279    ID=gene:YPR200C  Name=ARR2  ARR2    YPR200C
5159        XVI    939922    ID=gene:YPR201W  Name=ARR3  ARR3    YPR201W

[5160 rows x 6 columns]
#  number of genes: 5160


In [14]:

def  relabel_matrix(matrix, index_name_dict, full_gene_list):
    """ Function doc 
    
        Use dictionnary to relabel rows and columns of matrix.
        Intended to be used for instance with genes.
        
        Input:
            matrix:     (regulatory) matrix with row and column labels of genes or transcription factors (TF). 
            index_name_dict:    dictionnary to map TF names to gene names (obtained from ensembl library).
        
        Output:
            sub_matrix: a submatrix of the input matrix, that contains 
                        the rows and columns with labels
                        that are compatible with the index_name_dict.
    """
    old_cols = matrix.columns
    old_rows = matrix.index.values
    print('old_cols, n=',len(old_cols) )
    print(old_cols)
    print('old_rows, n=', len(old_rows))
    print(old_rows)
    print('#  parsing columns:')
    col_subset, new_cols_dict = select_compatible_labels(old_cols, index_name_dict, full_gene_list)
    print('#  parsing rows:')
    row_subset, new_rows_dict = select_compatible_labels(old_rows, index_name_dict, full_gene_list)
    sub_matrix = matrix.loc[ row_subset, col_subset ]
    sub_matrix.rename( columns = new_cols_dict, inplace=True)
    sub_matrix.rename( index = new_rows_dict, inplace=True)
    print('  number of compatible columns',len(col_subset))
    print('  number of compatible rows',len(row_subset))
    return sub_matrix


def  select_compatible_labels( old_labels, label_dict, full_label_list ):
    """ Function doc 
    
        Parse all old_labels to find those compatible with the label_dict.
        Incompatible labels are printed in output for logging.
        
        Input: 
            old_labels:     labels of row or column.
            label_dict:     dictionnary to map TF names to gene names (obtained from ensembl library).
    
        Output:
            label_subset:   subset of compatible labels.
            new_labels_dict:    dictionnary of compatible TF names to gene names.
    """
    label_subset=[]
    new_labels_dict={}
    
    for la in  old_labels:
        # The below line tested if the dash character was causing labels to be misclassified.
        li=la.replace(',','%2')
        
        label_x = li.upper()
        if li[-1]=='p':
            #   Note: this is for transcription factors (TF) from yeastract, which end in letter p.
            upper_li=li[:-1]
            label_x = upper_li.upper()
            
        if label_x in label_dict.keys() :
            label_subset.append( li )
            new_labels_dict[li] = label_dict[ label_x]
        
        elif label_x in full_label_list :
            label_subset.append( li )
            new_labels_dict[li] = label_x
            
        else:
            #   print output for log.
            print(' This label is incompatible:', li)            
    return label_subset, new_labels_dict
    

In [15]:
compatible_yeastract  =  relabel_matrix(  yeastract, gene_name_dict, full_gene_list )

old_cols, n= 3912
Index(['PAU8', 'SEO1', 'YAL064W-B', 'TDA8', 'YAL064W', 'FLO9', 'NHP6B',
       'YBR090C', 'TIM12', 'PHO3',
       ...
       'YBR012W-B', 'YDR210C-D', 'YGR161W-C', 'YHR214C-B', 'YPR158W-B',
       'MAL61', 'MAL62', 'Uncharacterized.167', 'YDR524W-C', 'MAL63'],
      dtype='object', length=3912)
old_rows, n= 152
['Smp1p' 'Ert1p' 'Thi2p' 'Mal33p' 'Kar4p' 'Hcm1p' 'Tup1p' 'Rds1p' 'Uga3p'
 'Pho2p' 'Mbp1p' 'Rpn4p' 'Nrg1p' 'Pdc2p' 'Ino2p' 'Swi5p' 'Ume6p' 'Upc2p'
 'Adr1p' 'Met32p' 'Yap6p' 'Sum1p' 'Aro80p' 'Cad1p' 'Yhp1p' 'Stp1p' 'Urc2p'
 'Gcn4p' 'Mig3p' 'Gln3p' 'Aca1p' 'Flo8p' 'Swi4p' 'Rph1p' 'Tog1p' 'Hac1p'
 'Gat1p' 'Pho4p' 'Fzf1p' 'Rtg2p' 'Hap2p' 'Mig2p' 'Cup2p' 'Tos8p' 'Hsf1p'
 'Aft1p' 'Mig1p' 'Pdr1p' 'Rme1p' 'Mga1p' 'Rim101p' 'Opi1p' 'Yap3p' 'Stp2p'
 'Ste12p' 'Ndt80p' 'Stb5p' 'Skn7p' 'Fkh1p' 'Xbp1p' 'Cst6p' 'Met28p'
 'Yap5p' 'Dal81p' 'Mga2p' 'YJL206C' 'Gzf3p' 'Cbf1p' 'Ime1p' 'Rsf2p'
 'Hms2p' 'Ash1p' 'Abf1p' 'Hap4p' 'Msn4p' 'Phd1p' 'Rgt1p' 'Ixr1p' 'Spt23p'
 'Put3p' 'Dal80p

In [16]:
compatible_yeastract

,YAL068C,YAL067C,YAL064W-B,YAL064C-A,YAL064W,YAL063C,YBR089C-A,YBR090C,YBR091C,YBR092C,...,YPR036W-A,YPR158W-A,YBR012W-A,YHR047C,YBR012W-B,YDR210C-D,YGR161W-C,YHR214C-B,YPR158W-B,YDR524W-C
3912,,,,,,,,,,,,,,,,,,,,,
YBR182C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR239C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR240C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR297W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YCL055W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YBL005W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR049C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YBR083W,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0


In [18]:
### remove columns whose names are not there in the expression matrix ######### 
expression_genes=pan.read_csv('/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/list_genes_in_order_01.csv',header=None)[0].to_list()
commoncolumns= [item for item in expression_genes if item in compatible_yeastract.columns]
commonrows=[item for item in expression_genes if item in compatible_yeastract.index]
compatible_yeastract= compatible_yeastract.loc[commonrows,commoncolumns]

In [19]:
print("Common Rows",len(commonrows))
print("Common columns",len(commoncolumns))

Common Rows 148
Common columns 3422


In [20]:
df=pan.read_csv('/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/strongest_eqtls_r_3columns.csv')
df.drop(columns=df.columns[0], axis=1,  inplace=True)
df.columns=['cis','pmarker','r']

In [21]:
df

,cis,pmarker,r
0,YAL003W,chrI:133174_G/A,0.383781
1,YAL008W,chrI:136961_T/C,0.273774
2,YAL009W,chrI:132723_G/A,-0.260297
3,YAL010C,chrI:134219_C/T,0.196381
4,YAL011W,chrI:131539_C/T,-0.302313
...,...,...,...
2879,YPR180W,chrXVI:897736_G/A,0.229310
2880,YPR186C,chrXVI:907815_T/C,0.131461
2881,YPR191W,chrXVI:921485_T/C,0.146219
2882,YPR192W,chrXVI:922297_T/C,0.790686


In [22]:
eqtlgenes=df['cis']

In [23]:
yeastractgenes=compatible_yeastract.index

In [24]:
len(yeastractgenes)
len(set(eqtlgenes).intersection(set(yeastractgenes)))  ## the number of trios that we can take is 80*3671 # 80 of the TFs are cis eQTLs
len(set(eqtlgenes).intersection(set(compatible_yeastract.columns))) ## 1710 out of the total 3671 columns genes are cis eQTLs 
len(set(yeastractgenes).intersection(set(compatible_yeastract.columns))) ## 125 TFs are both rows and columns, should we remove self regulation ?

123

In [25]:
#common tfs present in both yeastract and ciseqtl file
commontfs = [item for item in eqtlgenes if item in yeastractgenes] # not using set, because reproducing the result is not possible


In [34]:
#self regulations not removed, remove them while processing ? 
#cit gives NA values if both A and B are the same

In [35]:
#save common tfs
np.savetxt("commontfs.csv",
        commontfs,
        delimiter =", ",
        fmt ='% s')
#commontfs_test=pan.read_csv('commontfs.csv',header=None)[0].to_list()

In [26]:
#take only those rows that are common 
subset_yeast=compatible_yeastract.loc[commontfs,]

In [ ]:
######### Do a wilcoxin test unpaired #############

In [27]:
subsetrownames=subset_yeast.index
subsetcolnames=subset_yeast.columns

In [28]:
### create cis genotype dictionary ##### 
cisgeno={}
for i in range(0,len(df)):
    cisgeno[df['cis'][i]]=df['pmarker'][i]

In [30]:
# read the expression and genotype data 
expression_data=pan.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/expression_reordered_01.csv",index_col=0,header=0)
genotype=pan.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/genotypes_binary_strongest_eqtl.csv",header=0,index_col=0)

In [ ]:
##set seed## 
#seed generated using the following command 
# dont run it now, already used 
#import random
#import time
# Generate a seed using current time
#seed = int(time.time()) 
# seed = 1688557140 


In [31]:
#set the seed and dont set the seed while calling the sample function, it will reproduce the same results
np.random.seed( 1688557140 )

In [38]:
causal=[]
indep=[]
for j in range(0,len(subset_yeast)):
    # go through each row 
    groundtruth=subset_yeast.iloc[j,]
    Aname=subsetrownames[j]
    indice_list=[]
    L=genotype.loc[:,cisgeno[Aname]]
    #going to take only those who pass the wilkoxin test
    unique_values = np.unique(L)
    for value in unique_values:
        indices = np.where(L == value)[0]
        indice_list.append(indices)

    pvals=[]
    cols_used=[]
    for i in range(0,len(subsetcolnames)):
        Bname=subsetcolnames[i]
        if Aname==Bname: # remove self regulations
            continue
        cols_used.append(i)
        Bexpression=expression_data.loc[:,Bname]
    
        datasplit=[]
    
        datasplit.append(Bexpression[indice_list[0]])
        datasplit.append(Bexpression[indice_list[1]])
        pvals.append(ranksums(datasplit[0],datasplit[1])[1])
    pvals=multipletests(pvals, alpha=0.05, method='fdr_bh')[1] 
    comparison = [val < 0.2 for val in pvals]
    comp=pan.DataFrame({'colnames':subsetcolnames[cols_used].to_list(),'groundtruth':groundtruth[cols_used].to_list(),'pvalue':comparison})
    gt1=comp[(comp['groundtruth']==1) & (comp['pvalue'] == True)]
    gt1.insert(0,'A',Aname)
    gt1.insert(0,'L',cisgeno[Aname])
    causal.extend(gt1[['L','A','colnames','groundtruth']].values.tolist())
    # sample equal number of inpendent trios
    #seed set 
    assert len(gt1)<=len(comp[(comp['groundtruth']==0) & (comp['pvalue'] == True)])
    gt2=comp[(comp['groundtruth']==0) & (comp['pvalue'] == True)].sample(n=len(gt1))   #,random_state=1688557140)
    gt2.insert(0,'A',Aname)
    gt2.insert(0,'L',cisgeno[Aname])
    indep.extend(gt2[['L','A','colnames','groundtruth']].values.tolist())

    

In [39]:
causal_wilko=pan.DataFrame(causal,columns=['pmarker','cis','targetgene','groundtruth'])

In [40]:
len(causal_wilko)

1234

In [41]:
indp_wilko=pan.DataFrame(indep,columns=['pmarker','cis','targetgene','groundtruth'])

In [42]:
len(indp_wilko)

1234

In [65]:
## for yeast all causal data ########
#1257 was with self-regulations 
#1234 is without self-regulations
f = open("yeastgt_1_wilko1234_ready.txt", "w")
for i in range(len(causal_wilko)):
    sample=causal_wilko.iloc[i,:]
    L=genotype.loc[:,sample['pmarker']]
    A=expression_data.loc[:,sample['cis']]
    B=expression_data.loc[:,sample['targetgene']]
    trioheader= sample['pmarker']+' '+sample['cis']+' '+sample['targetgene']
    f.write(trioheader)
    f.write("\n")
    np.savetxt(f,L,newline=' ',fmt='%s')
    f.write("\n")
    np.savetxt(f,A,newline=' ',fmt='%s')
    f.write("\n")
    np.savetxt(f,B,newline=' ',fmt='%s')
    f.write("\n")
f.close()

In [66]:
## for yeast all independent data ########
f = open("yeastgt_0_wilko1234_ready.txt", "w")
for i in range(len(indp_wilko)):
    sample=indp_wilko.iloc[i,:]
    L=genotype.loc[:,sample['pmarker']]
    A=expression_data.loc[:,sample['cis']]
    B=expression_data.loc[:,sample['targetgene']]
    trioheader= sample['pmarker']+' '+sample['cis']+' '+sample['targetgene']
    f.write(trioheader)
    f.write("\n")
    np.savetxt(f,L,newline=' ',fmt='%s')
    f.write("\n")
    np.savetxt(f,A,newline=' ',fmt='%s')
    f.write("\n")
    np.savetxt(f,B,newline=' ',fmt='%s')
    f.write("\n")
f.close()